In [1]:
# %cd multi-object-tracker
# from motrackers import CentroidTracker, IOUTracker, CentroidKF_Tracker, SORT

In [1]:
from IPython.display import clear_output

import cv2
import numpy as np
import math

In [2]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [3]:

class Subtraction_Detector:
    'This class for car detection'
    def __init__(self, ):
        self.first_frame = None
    
    def detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5,5), 0) 
        
        if self.first_frame is None:
            self.first_frame = gray
            return np.array([]).reshape(0,4)
        
        list_objects = []
        
        mask = cv2.absdiff(self.first_frame, gray)
        mask = cv2.dilate(mask, np.ones((3,3)), iterations=3)
        _, mask = cv2.threshold(mask, 15, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 6000:
                x, y, w, h = cv2.boundingRect(cnt)
                list_objects.append([x,y,w,h])
        return np.array(list_objects)

def is_in_track_region(region, c):
    '''
    region in order: [x1,y1,x2,y2]
    c in order: [c_x, c_y]
    '''
    x1,y1,x2,y2 = region
    return (x1 <= c[0] <= x2) and (y1 <= c[1] <= y2) 

def to_city(track_info, x_l, x_r):
    """"
    check car's direction to the city or not
    """
    x,_,_ = track_info
    return x_r - x < x - x_l


In [4]:

cap = cv2.VideoCapture("./Traffic_Laramie_1.mp4")
num_cars = 0
detector = Subtraction_Detector()
tracker = EuclideanDistTracker()
bounding_x, bounding_y, bounding_w, bounding_h = 1,300,1035,300

# track_dict to Track object in my rectangle region
track_dict = {} 
tracking_region = [640, 30, 660, 300] # [x1,y1,x2,y2]
frame_id = 0
while True:
    ret, frame = cap.read()
    line_color = (0,0,130)
    if not ret:
        break
    height, width, _ = frame.shape
    roi = frame[300: 600,0: 1040]
    cv2.putText(frame, 'Main road', (50, 290), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    
    list_objects = detector.detect(roi)
    output_track = tracker.update(list_objects)

    for track in output_track:
        x1, y1, w, h, id = track
        cv2.rectangle(roi,(x1,y1),(x1+w,y1+h),(0,255,0),2)
        #cv2.putText(roi, f'car: {id}', (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,255), 2)
        
        cx,cy = int((2*x1 + w)/2), int((2*y1+h)/2)
        cv2.circle(roi, (cx,cy), 5, (0,0,255), -1)
        if is_in_track_region(tracking_region, (cx,cy)):
            track_dict[id] = track_dict.get(id, []) + [(cx,cy, frame_id)]
        
    
    out_keys = [k for k, v in track_dict.items() if v[-1][2] != frame_id]
    for k in out_keys:
        if to_city(track_info=track_dict[k][0], x_l=tracking_region[0], x_r=tracking_region[2]):
            num_cars += 1
            line_color = (0,0,255)
        del track_dict[k]
    
    cv2.rectangle(frame, (bounding_x, bounding_y), (bounding_x + bounding_w, bounding_y + bounding_h), (0,0,255), 2)
    cv2.line(roi, (tracking_region[0], tracking_region[1]), (tracking_region[0], tracking_region[3]), line_color, 5)
    cv2.putText(frame, f'frame: {frame_id}', (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.putText(frame, f'num cars: {num_cars}', (width-150, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,0,255), 1)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(30)
    if key == ord('q'):
        break
    
    frame_id += 1
    print(f'num_cars: {num_cars}')

cap.release()
cv2.destroyAllWindows()
print(num_cars)
clear_output()

In [12]:

# cap = cv2.VideoCapture("../Traffic_Laramie_1.mp4")
# num_cars = 0
# detector = Subtraction_Detector()
# tracker = CentroidKF_Tracker()
# bounding_x, bounding_y, bounding_w, bounding_h = 1,300,1035,300
# track_dict = {}
# tracking_region = [640, 30, 660, 300] # [x1,y1,x2,y2]
# frame_id = 0

# while True:
#     ret, frame = cap.read()
#     line_color = (0,0,130)
#     if not ret:
#         break
#     height, width, _ = frame.shape
#     roi = frame[300: 600,0: 1040]
    
#     list_objects = detector.detect(roi)
#     output_track = tracker.update(list_objects, np.ones(len(list_objects), np.ones(len(list_objects))))

#     for track in output_track:
#         _, id, x1, y1, w, h, _, _, _, _ = track
#         cv2.rectangle(roi,(x1,y1),(x1+w,y1+h),(0,255,0),2)
#         cv2.putText(roi, str(id), (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
#         cx,cy = int((2*x1 + w)/2), int((2*y1+h)/2)
#         cv2.circle(roi, (cx,cy), 5, (0,0,255), -1)
#         if is_in_track_region(tracking_region, (cx,cy)):
#             track_dict[id] = track_dict.get(id, []) + [(cx,cy, frame_id)]
        
    
#     out_keys = [k for k, v in track_dict.items() if v[-1][2] != frame_id]
#     for k in out_keys:
#         if to_city(track_info=track_dict[k][0], x_l=tracking_region[0], x_r=tracking_region[2]):
#             num_cars += 1
#             line_color = (0,0,255)
#         del track_dict[k]
    
#     cv2.rectangle(frame, (bounding_x, bounding_y), (bounding_x + bounding_w, bounding_y + bounding_h), (0,0,255), 1)
#     cv2.line(roi, (tracking_region[0], tracking_region[1]), (tracking_region[0], tracking_region[3]), line_color, 5)
#     cv2.putText(frame, f'car: {frame_id}', (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
#     cv2.putText(frame, f'num cars: {num_cars}', (width-150, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 1)
#     cv2.imshow("Frame", frame)

#     key = cv2.waitKey(30)
#     if key == ord('q'):
#         break
    
#     frame_id += 1
# cap.release()
# cv2.destroyAllWindows()

# clear_output()